# Self-Organized Criticality

Port of [Think Complexity chapter 8](http://greenteapress.com/complexity2/html/index.html) by Allen Downey.

In [ ]:
using Luxor
using Plots

## Critical Systems

Many critical systems demonstrate common behaviors:

- Fractal geometry: For example, freezing water tends to form fractal patterns, including snowflakes and other crystal structures. Fractals are characterized by self-similarity; that is, parts of the pattern are similar to scaled copies of the whole.

- Heavy-tailed distributions of some physical quantities: For example, in freezing water the distribution of crystal sizes is characterized by a power law.

- Variations in time that exhibit pink noise: Complex signals can be decomposed into their frequency components. In pink noise, low-frequency components have more power than high-frequency components. Specifically, the power at frequency f is proportional to 1/f.

Critical systems are usually unstable. For example, to keep water in a partially frozen state requires active control of the temperature. If the system is near the critical temperature, a small deviation tends to move the system into one phase or the other.

## Sand Piles

The sand pile model was proposed by Bak, Tang and Wiesenfeld in 1987. It is not meant to be a realistic model of a sand pile, but rather an abstraction that models physical systems with a large number of elements that interact with their neighbors.

The sand pile model is a 2-D cellular automaton where the state of each cell represents the slope of a part of a sand pile. During each time step, each cell is checked to see whether it exceeds a critical value, `K`, which is usually 3. If so, it “topples” and transfers sand to four neighboring cells; that is, the slope of the cell is decreased by 4, and each of the neighbors is increased by 1. At the perimeter of the grid, all cells are kept at slope 0, so the excess spills over the edge.

Bak, Tang and Wiesenfeld initialize all cells at a level greater than `K` and run the model until it stabilizes. Then they observe the effect of small perturbations: they choose a cell at random, increment its value by 1, and run the model again until it stabilizes.

For each perturbation, they measure `T`, the number of time steps the pile takes to stabilize, and `S`, the total number of cells that topple.

Most of the time, dropping a single grain causes no cells to topple, so `T=1` and `S=0`. But occasionally a single grain can cause an avalanche that affects a substantial fraction of the grid. The distributions of `T` and `S` turn out to be heavy-tailed, which supports the claim that the system is in a critical state.

They conclude that the sand pile model exhibits “self-organized criticality”, which means that it evolves toward a critical state without the need for external control or what they call “fine tuning” of any parameters. And the model stays in a critical state as more grains are added.

In the next few sections I replicate their experiments and interpret the results.

## Implementation

In [ ]:
function applytoppling(array::Array{Int64, 2}, K::Int64=3)
    out = copy(array)
    (ydim, xdim) = size(array)
    numtoppled = 0
    for y in 2:ydim-1
        for x in 2:xdim-1
            if array[y,x] > K
                numtoppled += 1
                out[y-1:y+1,x-1:x+1] += [0 1 0;1 -4 1;0 1 0]
            end
        end
    end
    out[1,:] .= 0
    out[end, :] .= 0
    out[:, 1] .= 0
    out[:, end] .= 0
    out, numtoppled
end

In [ ]:
pile = [0 0 0 0 0 0 0;
        0 0 0 0 0 0 0;
        0 0 4 0 4 0 0;
        0 0 0 0 0 0 0;
        0 0 0 0 0 0 0;]
pile, num = applytoppling(pile)
pile

In [ ]:
function visualizepile(array::Array{Int64, 2}, dim, scale)
    (ydim, xdim) = size(array)
    width = dim * (xdim - 1)
    height = dim * (ydim - 1)
    Drawing(width, height, "out.svg")
    for (j, y) in enumerate(2:ydim-1)
        for (i, x) in enumerate(2:xdim-1)
            sethue(setgrey(1-array[y,x]/scale))
            box(i*dim, j*dim, dim, dim, :fill)
        end
     end
     finish()
     preview()
end
visualizepile(pile, 30, 4)

In [ ]:
function steptoppling(array::Array{Int64, 2}, K::Int64=3)
    total = 0
    i = 0
    while true
        array, numtoppled = applytoppling(array, K)
        total += numtoppled
        i += 1
        if numtoppled == 0
            return array, i, total
        end
    end
end

In [ ]:
pile = zeros(Int64, 22, 22)
pile[2:end-1, 2:end-1] = 10 * ones(Int64, 20, 20)
visualizepile(pile, 15, 10)
pile, steps, total = steptoppling(pile)
visualizepile(pile, 15, 10)

With an initial level of 10, this sand pile takes 332 time steps to reach equilibrium, with a total of 53,336 topplings. The figure shows the configuration after this initial run. Notice that it has the repeating elements that are characteristic of fractals. We’ll come back to that soon.

In [ ]:
function drop(array::Array{Int64, 2})
    (ydim, xdim) = size(array)
    y = rand(2:ydim-1)
    x = rand(2:xdim-1)
    array[y,x] += 1
    array
end

In [ ]:
function runtoppling(array::Array{Int64, 2}, iter=200)
    array, steps, total = steptoppling(array, 3)
    for _ in 1:iter
        array = drop(array)
        array, steps, total = steptoppling(array, 3)
    end
    array
end

In [ ]:
pile = zeros(Int64, 22, 22)
pile[2:end-1, 2:end-1] = 10 * ones(Int64, 20, 20)
pile, steps, total = steptoppling(pile)
for i in 1:200
    pile = drop(pile)
    pile, steps, total = steptoppling(pile)
    if mod(i, 10) == 0
        visualizepile(pile, 15, 10)
        sleep(1)
    end
end

The figure shows the configuration of the sand pile after dropping 200 grains onto random cells, each time running until the pile reaches equilibrium. The symmetry of the initial configuration has been broken; the configuration looks random.

In [ ]:
for i in 1:200
    pile = drop(pile)
    pile, steps, total = steptoppling(pile)
end
visualizepile(pile, 15, 10)

Finally the figure shows the configuration after 400 drops. It looks similar to the configuration after 200 drops. In fact, the pile is now in a steady state where its statistical properties don’t change over time. I’ll explain some of those statistical properties in the next section.

## Heavy-Tailed Distributions

If the sand pile model is in a critical state, we expect to find heavy-tailed distributions for quantities like the duration and size of avalanches. So let’s take a look.

I’ll make a larger sand pile, with n=50 and an initial level of 30, and run until equilibrium:

In [ ]:
pile = zeros(Int64, 52, 52)
pile[2:end-1, 2:end-1] = 30 * ones(Int64, 50, 50)
pile, steps, total = steptoppling(pile)

Next, I’ll run 100,000 random drops

In [ ]:
durations = Int64[]
avalanches = Int64[]
for i in 1:100000
    pile = drop(pile)
    pile, steps, total = steptoppling(pile)
    push!(durations, steps)
    push!(avalanches, total)
end

A large majority of drops have duration 1 and no toppled cells; if we filter them out before plotting, we get a clearer view of the rest of the distribution.

In [ ]:
durations = filter(steps->steps>1, durations)
avalanches = filter(total->total>0, avalanches);

We build a histogram with the durations/avalanches as keys and their occurences as values.

In [ ]:
function hist(array)
    h = Dict()
    for v in array
        h[v] = get!(h, v, 0) + 1
    end
    h
end

We plot the probabilities of each value of the durations / avalanches with loglog axes.

In [ ]:
h = hist(durations)
total = sum(values(h))
x = Int64[]
y = Float64[]
for i in 2:maximum(collect(keys(h)))
    v = get(h, i, 0)
    if v ≠ 0
        push!(x, i)
        push!(y, v/total)
    end
end
plot(x, y, xaxis=:log, yaxis=:log, label="Durations")
h = hist(avalanches)
total = sum(values(h))
x = Int64[]
y = Float64[]
for i in 1:maximum(collect(keys(h)))
    v = get(h, i, 0)
    if v ≠ 0
        push!(x, i)
        push!(y, v/total)
    end
end
plot!(x, y, xaxis=:log, yaxis=:log, label="Avalanches")
x = collect(1:10000)
plot!(x, 1 ./ x, label="slope -1")

For values between 1 and 100, the distributions are nearly straight on a log-log scale, which is characteristic of a heavy tail. The gray lines in the figure have slopes near -1, which suggests that these distributions follow a power law with parameters near α=1.

For values greater than 100, the distributions fall away more quickly than the power law model, which means there are fewer very large values than the model predicts. One possibility is that this effect is due to the finite size of the sand pile; if so, we might expect larger piles to fit the power law better.

## Fractals

Another property of critical systems is fractal geometry. The initial configuration resembles a fractal, but you can’t always tell by looking. A more reliable way to identify a fractal is to estimate its fractal dimension, as we saw in previous lectures.

I’ll start by making a bigger sand pile, with `n=131` and initial level 22.

In [ ]:
pile = zeros(Int64, 133, 133)
pile[2:end-1, 2:end-1] = 22 * ones(Int64, 131, 131)
pile, steps, total = steptoppling(pile)
visualizepile(pile, 2, 10)

It takes 28,379 steps for this pile to reach equilibrium, with more than 200 million cells toppled.
To see the resulting pattern more clearly, I select cells with levels 0, 1, 2, and 3, and plot them separately:

In [ ]:
function visualizepileonekind(pile, dim, val)
    (ydim, xdim) = size(pile)
    width = dim * (xdim - 1)
    height = dim * (ydim - 1)
    Drawing(width, height, "out.svg")
    for (j, y) in enumerate(2:ydim-1)
        for (i, x) in enumerate(2:xdim-1)
            if pile[y,x] == val
                sethue("gray")
                box(i*dim, j*dim, dim, dim, :fill)
            end
        end
     end
     finish()
     preview()
end
visualizepileonekind(pile, 2, 3) # 0, 1, 2, 3

Visually, these patterns resemble fractals, but looks can be deceiving. To be more confident, we can estimate the fractal dimension for each pattern using box-counting.

We’ll count the number of cells in a small box at the center of the pile, then see how the number of cells increases as the box gets bigger. Here’s my implementation:

In [ ]:
function countcells(pile, val)
    (ydim, xdim) = size(pile)
    ymid = Int((ydim+1)/2)
    xmid = Int((xdim+1)/2)
    res = Int64[]
    for i in 0:Int((ydim-1)/2)-1
        push!(res, 1.0*count(x->x==val, pile[ymid-i:ymid+i,xmid-i:xmid+i]))
    end
    res
end

In [ ]:
level = 0
(ydim, xdim) = size(pile)
m = Int((ydim-1)/2)
res = filter(x->x>0, countcells(pile, level))
n = length(res)
plot(1:2:2*m-1, 1:2:2*m-1, xaxis=:log, yaxis=:log, label="d = 1")
plot!(1:2:2*m-1, (1:2:2*m-1).^2, xaxis=:log, yaxis=:log, label="d = 2")
plot!(1+2*(m-n):2:2*m-1, res, xaxis=:log, yaxis=:log, label="level $level")
level = 1
res = filter(x->x>0, countcells(pile, level))
n = length(res)
plot!(1+2*(m-n):2:2*m-1, res, xaxis=:log, yaxis=:log, label="level $level")
level = 2
res = filter(x->x>0, countcells(pile, level))
n = length(res)
plot!(1+2*(m-n):2:2*m-1, res, xaxis=:log, yaxis=:log, label="level $level")
level = 3
res = filter(x->x>0, countcells(pile, level))
n = length(res)
plot!(1+2*(m-n):2:2*m-1, res, xaxis=:log, yaxis=:log, label="level $level", legend=:topleft)

On a log-log scale, the cell counts form nearly straight lines, which indicates that we are measuring fractal dimension over a valid range of box sizes.

To estimate the slopes of these lines, we have to fot a line to the data by linear regression

In [ ]:
function linres(x, y)
    n = length(x)
    mx = sum(x) / n
    my = sum(y) / n
    beta = sum((x.-mx).*(y.-my))/sum((x.-mx).^2)
    alfa = my - beta * mx
    alfa, beta
end

In [ ]:
level = 3
(ydim, xdim) = size(pile)
m = Int((ydim-1)/2)
res = filter(x->x>0, countcells(pile, level))
n = length(res)
linres(log.(1.0*collect(1+2*(m-n):2:2*m-1)), log.(res))

The estimated fractal dimensions are:
0. 1.868
1. 3.494
2. 1.784
3. 2.081

The fractal dimension for levels 0, 1, and 2 seems to be clearly non-integer, which indicates that the image is fractal.
The estimate for level 3 is indistinguishable from 2, but given the results for the other values, the apparent curvature of the line, and the appearance of the pattern, it seems likely that it is also fractal.